<a href="https://colab.research.google.com/github/janbanot/msc-project/blob/main/test_notebooks/test_pharaphrases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Parafraz T5

Ten notatnik testuje jakość parafraz generowanych przez model T5 (Vamsi/T5_Paraphrase_Paws).

**Cel:**
- Wczytać 20 toksycznych przykładów z datasetu
- Wygenerować parafrazy używając T5
- Porównać oryginał vs parafraza
- Wyświetlić metryki jakości (cosine similarity, zachowanie toksyczności)

In [ ]:
!uv pip install transformers datasets torch pandas matplotlib seaborn

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
)

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 1000)

In [ ]:
from google.colab import drive
drive.mount('/drive')

In [ ]:
# ===================================================
# KONFIGURACJA
# ===================================================

# Parametry
N_SAMPLES = 20  # Liczba próbek do testowania
MAX_SEQUENCE_LENGTH = 256  # Maksymalna długość sekwencji
TARGET_LAYER_INDEX = 5  # Warstwa do ekstrakcji reprezentacji
PARAPHRASE_SEED = 42  # Seed dla reproducibility
PARAPHRASE_MIN_SIMILARITY = 0.7  # Minimalny próg jakości parafrazy

# Ścieżki
DATA_PATH = "/drive/MyDrive/msc-project/jigsaw-toxic-comment/train.csv"
MODEL_CHECKPOINT = "/drive/MyDrive/msc-project/models/distilbert-jigsaw-full"

# Urządzenie
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Uruchomiono na urządzeniu: {device}")

# Ustawienie seed
torch.manual_seed(PARAPHRASE_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(PARAPHRASE_SEED)

In [ ]:
# ===================================================
# ŁADOWANIE MODELI
# ===================================================

print(">>> Ładowanie modelu klasyfikacji toksyczności...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT, num_labels=1, problem_type="single_label_classification"
    )
    model.to(device)
    model.eval()
    print("✓ Model DistilBERT załadowany")
except Exception as e:
    print(f"✗ Błąd ładowania modelu: {e}")
    raise

print("\n>>> Ładowanie modelu T5 do generowania parafraz...")
t5_name = "Vamsi/T5_Paraphrase_Paws"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_name)
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_name).to(device)
t5_model.eval()
print("✓ Model T5 załadowany")

print("\n>>> Modele gotowe!")

In [ ]:
# ===================================================
# ŁADOWANIE DANYCH
# ===================================================

def clean_text(text):
    """Czyści tekst (zgodnie z preprocessing z głównego notatnika)."""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", text)
    text = re.sub(r"\(talk\)", "", text)
    text = re.sub(r"\d{2}:\d{2}, \w+ \d{1,2}, \d{4} \(utc\)", "", text)
    text = text.replace("\n", " ").replace("\xa0", " ")
    text = text.strip(' "')
    text = re.sub(r"\s+", " ", text).strip()
    return text

print(">>> Wczytywanie danych...")
try:
    df = pd.read_csv(DATA_PATH)
    print(f"✓ Wczytano {len(df)} rekordów")
    
    # Filtruj tylko toksyczne komentarze
    toxic_df = df[df['toxic'] == 1].copy()
    print(f"✓ Znaleziono {len(toxic_df)} toksycznych komentarzy")
    
    # Wybierz N_SAMPLES losowych próbek
    sample_df = toxic_df.sample(n=min(N_SAMPLES, len(toxic_df)), random_state=PARAPHRASE_SEED)
    
    # Wyczyść teksty
    sample_df['comment_text'] = sample_df['comment_text'].apply(clean_text)
    
    # Usuń puste teksty
    sample_df = sample_df[sample_df['comment_text'].str.len() > 10]
    
    print(f"✓ Wybrano {len(sample_df)} próbek do testowania")
    
except Exception as e:
    print(f"✗ Błąd wczytywania danych: {e}")
    raise

In [ ]:
# ===================================================
# FUNKCJE POMOCNICZE
# ===================================================

def get_embedding_and_prob(text):
    """
    Pobiera embedding [CLS] z warstwy TARGET_LAYER_INDEX oraz prawdopodobieństwo toksyczności.
    
    Zwraca:
        Tuple (embedding_vector, toxic_probability)
    """
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=MAX_SEQUENCE_LENGTH,
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        embedding = outputs.hidden_states[TARGET_LAYER_INDEX][0, 0, :]  # [CLS] token
        prob = torch.sigmoid(outputs.logits)[0, 0].item()
    
    return embedding, prob


def generate_paraphrase(text):
    """
    Generuje parafrazę używając modelu T5.
    
    Zwraca:
        Wygenerowana parafraza (string)
    """
    # Prefix "paraphrase:" jest wymagany przez ten model
    input_text = "paraphrase: " + text + " </s>"
    
    inputs = t5_tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_SEQUENCE_LENGTH,
    ).to(device)
    
    with torch.no_grad():
        outputs = t5_model.generate(
            inputs.input_ids,
            max_length=MAX_SEQUENCE_LENGTH,
            do_sample=True,
            top_k=50,
            num_return_sequences=1,
        )
    
    paraphrase = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return paraphrase


def calculate_cosine_similarity(emb1, emb2):
    """Oblicza cosine similarity między dwoma embeddingami."""
    return F.cosine_similarity(emb1.unsqueeze(0), emb2.unsqueeze(0)).item()

print("✓ Funkcje pomocnicze zdefiniowane")

In [ ]:
# ===================================================
# GENEROWANIE PARAFRAZ I ZBIERANIE WYNIKÓW
# ===================================================

print(">>> Rozpoczynam generowanie parafraz...\n")

results = []

for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="Generowanie parafraz"):
    original_text = row['comment_text']
    
    try:
        # 1. Oblicz embedding i prawdopodobieństwo dla oryginału
        orig_emb, orig_prob = get_embedding_and_prob(original_text)
        
        # 2. Wygeneruj parafrazę
        para_text = generate_paraphrase(original_text)
        
        # 3. Oblicz embedding i prawdopodobieństwo dla parafrazy
        para_emb, para_prob = get_embedding_and_prob(para_text)
        
        # 4. Oblicz cosine similarity
        cos_sim = calculate_cosine_similarity(orig_emb, para_emb)
        
        # 5. Oblicz różnicę prawdopodobieństw
        prob_diff = abs(orig_prob - para_prob)
        
        # 6. Sprawdź czy parafraza przeszła walidację
        quality_ok = cos_sim >= PARAPHRASE_MIN_SIMILARITY
        
        # Skróć teksty dla wyświetlenia (pierwsze 100 znaków)
        orig_display = original_text[:100] + "..." if len(original_text) > 100 else original_text
        para_display = para_text[:100] + "..." if len(para_text) > 100 else para_text
        
        results.append({
            'id': len(results) + 1,
            'original_text': orig_display,
            'paraphrase_text': para_display,
            'orig_prob': round(orig_prob, 3),
            'para_prob': round(para_prob, 3),
            'prob_diff': round(prob_diff, 3),
            'cosine_sim': round(cos_sim, 3),
            'quality_ok': '✓' if quality_ok else '✗',
        })
        
    except Exception as e:
        print(f"Błąd dla próbki {idx}: {e}")
        continue

print(f"\n✓ Przetworzono {len(results)} par tekst-parafraza")

In [ ]:
# ===================================================
# WYŚWIETLENIE TABELI WYNIKÓW
# ===================================================

results_df = pd.DataFrame(results)

print("="*100)
print("WYNIKI TESTOWANIA PARAFRAZ")
print("="*100)
print()

# Wyświetl pełną tabelę
print(results_df.to_string(index=False))
print()

# Zapisz do CSV
output_path = "/drive/MyDrive/msc-project/paraphrase_test_results.csv"
results_df.to_csv(output_path, index=False)
print(f"✓ Wyniki zapisane do: {output_path}")

In [ ]:
# ===================================================
# STATYSTYKI PODSUMOWUJĄCE
# ===================================================

print("\n" + "="*100)
print("STATYSTYKI PODSUMOWUJĄCE")
print("="*100)
print()

# Podstawowe statystyki
print(f"Liczba przetestowanych par: {len(results_df)}")
print()

# Jakość parafraz
quality_pass = (results_df['quality_ok'] == '✓').sum()
quality_fail = (results_df['quality_ok'] == '✗').sum()
quality_pass_pct = (quality_pass / len(results_df) * 100) if len(results_df) > 0 else 0

print(f"Parafrazy przechodzące walidację (cos_sim ≥ {PARAPHRASE_MIN_SIMILARITY}): {quality_pass} ({quality_pass_pct:.1f}%)")
print(f"Parafrazy nieprzechodzące walidacji: {quality_fail} ({100-quality_pass_pct:.1f}%)")
print()

# Zachowanie toksyczności
toxic_maintained = (results_df['para_prob'] > 0.5).sum()
toxic_lost = (results_df['para_prob'] <= 0.5).sum()
toxic_maintained_pct = (toxic_maintained / len(results_df) * 100) if len(results_df) > 0 else 0

print(f"Parafrazy zachowujące toksyczność (prob > 0.5): {toxic_maintained} ({toxic_maintained_pct:.1f}%)")
print(f"Parafrazy tracące toksyczność (prob ≤ 0.5): {toxic_lost} ({100-toxic_maintained_pct:.1f}%)")
print()

# Statystyki metryk
print("Statystyki metryk:")
print(f"  Cosine Similarity:")
print(f"    - Średnia: {results_df['cosine_sim'].mean():.3f}")
print(f"    - Min: {results_df['cosine_sim'].min():.3f}")
print(f"    - Max: {results_df['cosine_sim'].max():.3f}")
print(f"    - Std: {results_df['cosine_sim'].std():.3f}")
print()
print(f"  Różnica prawdopodobieństw:")
print(f"    - Średnia: {results_df['prob_diff'].mean():.3f}")
print(f"    - Min: {results_df['prob_diff'].min():.3f}")
print(f"    - Max: {results_df['prob_diff'].max():.3f}")
print(f"    - Std: {results_df['prob_diff'].std():.3f}")
print()

# Wnioski
print("="*100)
print("WNIOSKI")
print("="*100)
print()

if quality_pass_pct >= 80:
    print("✓ Wysoka jakość parafraz (≥80% przechodzi walidację)")
elif quality_pass_pct >= 60:
    print("⚠ Średnia jakość parafraz (60-80% przechodzi walidację)")
else:
    print("✗ Niska jakość parafraz (<60% przechodzi walidację)")

if toxic_maintained_pct >= 80:
    print("✓ Parafrazy dobrze zachowują toksyczność (≥80%)")
elif toxic_maintained_pct >= 60:
    print("⚠ Parafrazy średnio zachowują toksyczność (60-80%)")
else:
    print("✗ Parafrazy tracą toksyczność (<60% zachowuje)")

if results_df['cosine_sim'].mean() >= 0.8:
    print("✓ Wysokie podobieństwo semantyczne (średnia ≥0.8)")
elif results_df['cosine_sim'].mean() >= 0.7:
    print("⚠ Średnie podobieństwo semantyczne (średnia 0.7-0.8)")
else:
    print("✗ Niskie podobieństwo semantyczne (średnia <0.7)")

In [ ]:
# ===================================================
# PRZYKŁADY (Pierwsze 5 par)
# ===================================================

print("\n" + "="*100)
print("PRZYKŁADOWE PARY TEKST-PARAFRAZA (pierwsze 5)")
print("="*100)
print()

for i, row in results_df.head(5).iterrows():
    print(f"--- Przykład {row['id']} ---")
    print(f"ORYGINAŁ: {row['original_text']}")
    print(f"PARAFRAZA: {row['paraphrase_text']}")
    print(f"Prawdop. toksyczności: {row['orig_prob']} → {row['para_prob']} (diff: {row['prob_diff']})")
    print(f"Cosine similarity: {row['cosine_sim']} {row['quality_ok']}")
    print()